In [1]:
import os
import sys
import glob
import random
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data  import DataLoader, Dataset
from ipywidgets import interact, widgets
import matplotlib.pyplot as plt
from PIL import Image
import numpy as np

%matplotlib inline

In [2]:
import models
from data import *

In [3]:
# ---------------------------------------------------------------
loadSize = 143
fineSize = 128

# Number of G, D Features
ngf = 64
ndf = 64    
# Number of Input and output Channels
in_nc = 3 
out_nc = 3

cuda=False
# ---------------------------------------------------------------

### Image Transformations

In [4]:
# Image Transformations
transform = transforms.Compose([
    transforms.Resize(int(1000), Image.BICUBIC)
    , transforms.RandomCrop(1000)
    , transforms.ToTensor()
    , transforms.Normalize((0.5,0.5,0.5), (0.5,0.5,0.5)) 
])

In [5]:
# Init Generator
gen = models.Generator(in_nc, out_nc, 4, ngf)
gen.load_state_dict(torch.load('./model-states/netG100.pth', map_location='cpu'))

<All keys matched successfully>

In [6]:
def tensor2image(tensor):
    image = 127.5*(tensor[0].cpu().float().detach().numpy() + 1.0)
    return image.astype(np.uint8)

### Interactive Widget

In [7]:
@interact(vango=widgets.FloatSlider(min=0,max=1,step=.01,value=0,continuous_update=False)
          , ukyo=widgets.FloatSlider(min=0,max=1,step=.01,value=0,continuous_update=False)
          , monet=widgets.FloatSlider(min=0,max=1,step=.01,value=0,continuous_update=False)
          , cezan=widgets.FloatSlider(min=0,max=1,step=.01,value=0,continuous_update=False)
          , ident=widgets.FloatSlider(min=.01,max=1,step=.01,value=1,continuous_update=False) )
def generate_image(vango,ukyo,monet,cezan,ident
                   , path='./unknown_kiellinie.jpg'
                   , flip90=False):
    
    generator = gen
    content = transform(Image.open(path))
    Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor
    input_A = Tensor(1, 3, 1000, 1000)
    real_A = input_A.copy_(content)
    #print(style_label)
    
    generated = gen({
        'content': real_A,
        'style_label': [[cezan,monet,ukyo,vango,ident]]   
    })
    im=tensor2image(generated.data)
    if flip90:
        im=im.transpose(2,1,0)
    else:
        im=im.transpose(1,2,0)
    fig = plt.figure(figsize=(12,12))
    ax = fig.add_subplot(111)  
    ax.imshow(im)
    plt.savefig('./generated.png')

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='vango', max=1.0, step=0.01)…

***
Ref : @[GatedGAN](https://github.com/colemiller94/gatedgan)